In [1]:
# Celda 1

import pandas as pd                   # Importa la librería pandas para la manipulación de estructuras de datos (DataFrames)
import numpy as np                    # Importa numpy para operaciones numéricas y manejo de arrays
import matplotlib.pyplot as plt       # Importa matplotlib para la creación de gráficos
import yfinance as yf                 # Importa yfinance para descargar datos financieros desde Yahoo Finance
from statsmodels.tsa.seasonal import seasonal_decompose  # Importa la función para descomponer series de tiempo
from statsmodels.tsa.arima.model import ARIMA            # Importa el modelo ARIMA para análisis y predicción de series
import seaborn as sns  # Seaborn maneja los estilos de Matplotlib
sns.set_style("darkgrid")  # Aplica un estilo de cuadrícula oscura



In [2]:
# Celda 2

# Definimos el símbolo y el rango de fechas para descargar los datos
ticker_symbol = 'AAPL'                # Símbolo de la acción de Apple
start_date = '2020-01-01'             # Fecha de inicio para la descarga de datos
end_date = '2022-12-31'               # Fecha de finalización para la descarga de datos

# Descargamos los datos históricos de AAPL y los almacenamos en un DataFrame
df = yf.download(ticker_symbol, start=start_date, end=end_date)  # Función que descarga los datos de Yahoo Finance

# Mostramos las primeras 5 filas del DataFrame para verificar su contenido
df.head()                             # Imprime las primeras 5 filas del DataFrame


[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2020-01-02,72.716072,72.776598,71.466812,71.721019,135480400
2020-01-03,72.009117,72.771745,71.783962,71.941328,146322800
2020-01-06,72.582916,72.621654,70.876083,71.127873,118387200
2020-01-07,72.241531,72.849208,72.021215,72.592578,108872000
2020-01-08,73.403641,73.706271,71.943751,71.943751,132079200
